# Loan application summary

The following data sample is taken Kaggle (https://www.kaggle.com/wendykan/lending-club-loan-data). It contains more than 
800K customer data. 
The purpose of this file is to summarize the characterize of the lenders, model 
a classifier to determine a customer will default or not and determine to who to advertise.

In [32]:
import pandas as pd
import os
import pandas as pd
import numpy as np
import nltk
from sklearn.feature_extraction.text import CountVectorizer  
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.cluster import KMeans
from matplotlib import rcParams
import plotly.plotly as py
import plotly
import pandas as pd
from plotly.graph_objs import *
import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='otaniemi', api_key='xg2mMuVQ9CTWdbzP2CWM')

Read the csv file and have an overview summary of the document.

In [33]:
df_loan = pd.read_csv("loan.csv",low_memory=False)

In [34]:
print("Top ten loan Purposes : ")
print(df_loan.purpose.value_counts().nlargest(10))

Top ten loan Purposes : 
debt_consolidation    524215
credit_card           206182
home_improvement       51829
other                  42894
major_purchase         17277
small_business         10377
car                     8863
medical                 8540
moving                  5414
vacation                4736
Name: purpose, dtype: int64


In [35]:
print("Top ten Employments types : ")
print(df_loan.emp_title.value_counts().nlargest(10))

Top ten Employments types : 
Teacher             13469
Manager             11240
Registered Nurse     5525
Owner                5376
RN                   5355
Supervisor           4983
Sales                4212
Project Manager      3988
Driver               3569
Office Manager       3510
Name: emp_title, dtype: int64


In [36]:
print("The first 10 most frequency titles provided by Borrower")
print(df_loan.title.value_counts().nlargest(10))

The first 10 most frequency titles provided by Borrower
Debt consolidation         414001
Credit card refinancing    164331
Home improvement            40112
Other                       31892
Debt Consolidation          15760
Major purchase              12051
Business                     6728
Medical expenses             6674
Car financing                5565
Consolidation                5381
Name: title, dtype: int64


In [37]:
print(pd.crosstab(df_loan.term, df_loan.application_type))

application_type  INDIVIDUAL  JOINT
term                               
 36 months            620886    239
 60 months            265982    272


In [38]:
print("The change in interest rate over the years")
data = [go.Histogram(x=df_loan['int_rate'])]
py.iplot(data)

The change in interest rate over the years


C:\Users\fanuel\Anaconda3\lib\site-packages\plotly\plotly\plotly.py:224: UserWarning:

Woah there! Look at all those points! Due to browser limitations, the Plotly SVG drawing functions have a hard time graphing more than 500k data points for line charts, or 40k points for other types of charts. Here are some suggestions:
(1) Use the `plotly.graph_objs.Scattergl` trace object to generate a WebGl graph.
(2) Trying using the image API to return an image instead of a graph URL
(3) Use matplotlib
(4) See if you can create your visualization with fewer data points

If the visualization you're using aggregates points (e.g., box plot, histogram, etc.) you can disregard this warning.



The distribution of successful payment rate across states. The loan which are paid in full are classified as 1 while late payments and defaults are classified as 0. The loan staus with unknown future like 'Current' are ignored.

In [39]:
df1 = df_loan[['loan_status','addr_state']]
cat = ['loan_status']
for i in cat:
    df1[i] = df1[i].astype('category')
for i in cat:
    df1[i] = df1[i].cat.codes
d = { 5:1, 0:0, 9:0, 6:0, 3:0, 4:0, 2:0}
df1['loan_status'] = df1['loan_status'].map(d)
df1 = df1.dropna(axis=0, how='any')
 

C:\Users\fanuel\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\fanuel\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\fanuel\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#i

In [40]:
mean=df1.groupby('addr_state')['loan_status'].mean()
total=df1.groupby('addr_state')['loan_status'].count()
maps=[]
tmean=[]
ttotal=[]
state = df1['addr_state'].unique()
for i in state:
    tmean.append(mean[i])
    ttotal.append(total[i])
maps = pd.DataFrame(state, columns=list('S'))   
tmean = pd.DataFrame(tmean, columns=list('c')) 
ttotal = pd.DataFrame(ttotal, columns=list('d')) 



In [41]:
frames = [maps, tmean, ttotal]
df = pd.concat(frames, axis=1)

In [42]:
for col in df.columns:
    df[col] = df[col].astype(str)

scl = [[0.0, 'rgb(255, 0, 0)'],[0.2, 'rgb(255,208,0)'],[0.4, 'rgb(255,240,0)'],\
            [0.6, 'rgb(255,255,0)'],[0.8, 'rgb(127,255,0)'],[1.0, 'rgb(0,255,0)']]

df['text'] = df['S'] + '<br>' +\
    'Successful rate '+df['c']+'<br>'+\
    'Application number '+df['d']

data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = df['S'],
        z = df['c'].astype(float),
        locationmode = 'USA-states',
        text = df['text'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Successful payment")
        ) ]

layout = dict(
        title = 'Default rate by State<br>(Hover for breakdown)',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    
fig = dict( data=data, layout=layout )
py.iplot( fig, filename='d3-cloropleth-map' )

The loan application over the years

In [43]:
df_loan['month'], df_loan['year'] = df_loan['issue_d'].str.split('-', 1).str
j = df_loan['year'].value_counts(ascending=True)
print(j)
x=df_loan['year']
y=df_loan['year'].value_counts()
data = [go.Histogram(x=df_loan['year'])]
py.iplot(data)

2007       603
2008      2393
2009      5281
2010     12537
2011     21721
2012     53367
2013    134755
2014    235628
2015    421094
Name: year, dtype: int64


C:\Users\fanuel\Anaconda3\lib\site-packages\plotly\plotly\plotly.py:224: UserWarning:

Woah there! Look at all those points! Due to browser limitations, the Plotly SVG drawing functions have a hard time graphing more than 500k data points for line charts, or 40k points for other types of charts. Here are some suggestions:
(1) Use the `plotly.graph_objs.Scattergl` trace object to generate a WebGl graph.
(2) Trying using the image API to return an image instead of a graph URL
(3) Use matplotlib
(4) See if you can create your visualization with fewer data points

If the visualization you're using aggregates points (e.g., box plot, histogram, etc.) you can disregard this warning.



In [44]:
time = df_loan.year.unique()
time.sort()

In [45]:
print("The top demographics and purpose change over the years")
for i in time:
    demo = df_loan.loc[df_loan.year == i,:]
    print(i)
    print(demo.emp_title.value_counts().nlargest(2) )
    print(demo.purpose.value_counts().nlargest(2))
    print()


The top demographics and purpose change over the years
2007
Self Employed    4
Best Buy         3
Name: emp_title, dtype: int64
debt_consolidation    207
other                 113
Name: purpose, dtype: int64

2008
Self Employed    16
Self             15
Name: emp_title, dtype: int64
debt_consolidation    976
credit_card           399
Name: purpose, dtype: int64

2009
Self Employed    20
Retired          13
Name: emp_title, dtype: int64
debt_consolidation    2194
other                  659
Name: purpose, dtype: int64

2010
Bank of America    43
US Army            43
Name: emp_title, dtype: int64
debt_consolidation    5729
credit_card           1545
Name: purpose, dtype: int64

2011
US Army            78
Bank of America    65
Name: emp_title, dtype: int64
debt_consolidation    10670
credit_card            2820
Name: purpose, dtype: int64

2012
US Army            204
Bank of America    125
Name: emp_title, dtype: int64
debt_consolidation    30811
credit_card           10365
Name: purpose,

# Preprocessing

The intial data contains more than 40 rows. We check if more than 50 % of the values are NaN, we drop those feature. 

In [46]:
check_null = df_loan.isnull().sum(axis=0).sort_values(ascending=False)/float(len(df_loan))
check_null[check_null>0.5]

dti_joint                      0.999426
annual_inc_joint               0.999424
verification_status_joint      0.999424
il_util                        0.979020
mths_since_rcnt_il             0.976549
all_util                       0.975916
max_bal_bc                     0.975916
open_rv_24m                    0.975916
open_rv_12m                    0.975916
inq_last_12m                   0.975916
total_bal_il                   0.975916
inq_fi                         0.975916
open_il_24m                    0.975916
open_il_12m                    0.975916
open_il_6m                     0.975916
open_acc_6m                    0.975916
total_cu_tl                    0.975916
desc                           0.857976
mths_since_last_record         0.845553
mths_since_last_major_derog    0.750160
mths_since_last_delinq         0.511971
dtype: float64

In [47]:
df_loan.drop(check_null[check_null>0.5].index, axis=1, inplace=True) 
df_loan.dropna(axis=0, thresh=30, inplace=True)


There were no meta data or concrete information or the redundant nature in the following variables listed below and were dropped. The application type ratio is also statiscally insiginificant so it was also dropped. 

In [48]:
unused = ['zip_code',
          'sub_grade',
          'url',
         'id',
         'member_id',
         'dti',
         'delinq_2yrs',
         'earliest_cr_line',
         'inq_last_6mths',
         'open_acc',
         'pub_rec',
         'revol_bal',
         'revol_util',
         'total_acc',
         'initial_list_status',
         'out_prncp',
         'out_prncp_inv',
         'total_pymnt',
         'total_pymnt_inv',
         'total_rec_prncp',
         'total_rec_int',
         'total_rec_late_fee',
         'recoveries',
         'collection_recovery_fee',
         'last_pymnt_d',
         'last_pymnt_amnt',
         'next_pymnt_d',
         'last_credit_pull_d',
         'collections_12_mths_ex_med',
         'policy_code',
         'application_type',
         'acc_now_delinq',
         'tot_coll_amt',
         'tot_cur_bal',
         'total_rev_hi_lim',
         'emp_title',
         'issue_d',
         'purpose',
         'funded_amnt_inv',
         'title',
         'pymnt_plan']
df_loan.drop(unused , axis=1, inplace=True) 


In order to proceed with the classification task, we have vectorize all the data inside the table.

In [49]:
cat = ['term','grade','emp_length','home_ownership','verification_status','loan_status','addr_state']
for i in cat:
    df_loan[i] = df_loan[i].astype('category')
for i in cat:
    df_loan[i] = df_loan[i].cat.codes


The purpose of the classification task is to see if can sucessfully classify the good customers to those who default. The loan status indicates the condition of the loan. The loans which are in the "current" type and "issued" dont have any use as the final outcome is decided later in the future. It will be dropped. The rest of the loans are classified as good loan (Fully Paid and Late (16-30 days) ) and bad loan (the rest).

In [50]:
d = { 5:1, 0:0, 9:0, 6:0, 3:0, 4:0, 2:0}
df_loan['loan_status'] = df_loan['loan_status'].map(d)
e = { 1:11, 2:3, 10:1, 3:4, 0:2, 5:6, 4:5, 11:0, 7:8, 8:9, 6:7, 9:10}
df_loan['emp_length'] = df_loan['emp_length'].map(e)

In [51]:
loan = df_loan.dropna(axis=0, how='any')

The table below summarizes the correlation matrix between the variable. As expected the strong relations are:
   1. The loan amount and installment 
   2. The interest rate and credit rating 
   3. The loan amount and loan period

In [52]:
Corr = loan.corr()
Corr

,loan_amnt,funded_amnt,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,addr_state
loan_amnt,1.000000,0.997717,0.406008,0.215911,0.954630,0.208558,0.133776,-0.190120,0.324909,0.362980,-0.068610,0.011184
funded_amnt,0.997717,1.000000,0.402557,0.217592,0.958037,0.208541,0.134308,-0.189586,0.324029,0.362042,-0.068780,0.011404
term,0.406008,0.402557,1.000000,0.450981,0.170739,0.465074,0.081144,-0.103990,0.063538,0.257355,-0.164636,0.016975
int_rate,0.215911,0.217592,0.450981,1.000000,0.205929,0.946433,0.025623,0.067191,-0.025528,0.241834,-0.253271,0.006178
installment,0.954630,0.958037,0.170739,0.205929,1.000000,0.188372,0.120683,-0.160273,0.320865,0.334533,-0.052388,0.007451
grade,0.208558,0.208541,0.465074,0.946433,0.188372,1.000000,0.013840,0.064587,-0.016822,0.217286,-0.265658,0.005819
emp_length,0.133776,0.134308,0.081144,0.025623,0.120683,0.013840,1.000000,-0.198512,0.096555,0.021015,0.023450,0.008559
home_ownership,-0.190120,-0.189586,-0.103990,0.067191,-0.160273,0.064587,-0.198512,1.000000,-0.167065,-0.085053,-0.056693,-0.060898
annual_inc,0.324909,0.324029,0.063538,-0.025528,0.320865,-0.016822,0.096555,-0.167065,1.000000,0.097812,0.048770,-0.005246
verification_status,0.362980,0.362042,0.257355,0.241834,0.334533,0.217286,0.021015,-0.085053,0.097812,1.000000,-0.068292,-0.005538


In [53]:
sample = loan[10000:80000]
features = list(sample.columns[:9])
X = (sample[features])
pca = PCA(n_components=2, whiten=True).fit(X)
X_pca = pca.transform(X)
print(X_pca)

[[ 0.05329895  1.04992139]
 [-0.24787957  1.18448644]
 [ 0.20614362 -1.49767154]
 ..., 
 [-0.42286621  1.07544238]
 [ 1.39508451 -0.67243572]
 [ 0.57755256  1.43812237]]


The explained variance ratio is 99.99%

In [54]:
print (pca.explained_variance_ratio_)
print (sum(pca.explained_variance_ratio_))
print(X_pca.shape)

[ 0.96776127  0.03209427]
0.999855542076
(70000, 2)


The same 70,000 dataset is used to train and test the model. The test size will be selected randomly and it represents 20 %. The classification method used is Decision tree. It is chosen because conditional control statements applied in the decision making.

In [55]:

train_x, test_x, train_y, test_y =train_test_split(X_pca, sample["loan_status"],
                                                   test_size=0.2, random_state=42)


In [56]:
clf = tree.DecisionTreeClassifier()


In [57]:
y_score =clf.fit(train_x, train_y)
score = clf.score(test_x, test_y)
print(score)

0.659928571429


In [58]:
y_pred = clf.predict(test_x)
cnf_matrix = confusion_matrix(test_y, y_pred)
print("The confusion matrix")
print(cnf_matrix)

The confusion matrix
[[ 900 2306]
 [2455 8339]]


The models has a 65 % accuracy. It miss classifies 17 % people as good lenders while they are expected to default. Considering the typical default rate is 30%. It is safe to conclude we have reduced the deault rate by half. https://www.thebalance.com/credit-card-default-and-penalty-rates-explained-960643 

# Summary

We are able to summarize the data by location, job application and purpose. In the early times, the dominant demographics who are good customer (Fully paid on time) were entrepreneurs it then switched to Army personnel and in rececent years, it has changed to Teachers and managers. All over the years the most common purpose of the loan is debt consolidation. In terms of geographic location, the unregularized map shows the places with lowest default is Maine. It also shows the success payment in addition with the application number. 